# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

from datetime import datetime

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City ID.1
0,0,kirakira,-10.4544,161.9205,27.12,84,100,1.88,SB,2024-04-02,2178753
1,1,pangody,65.8500,74.4859,0.15,98,100,8.69,RU,2024-04-02,1495626
2,2,chul'man,56.8479,124.9104,-6.37,92,99,4.85,RU,2024-04-02,2025261
3,3,nieuw amsterdam,5.8833,-55.0833,29.01,70,20,1.03,SR,2024-04-02,3383434
4,4,dudinka,69.4058,86.1778,-10.18,94,100,6.71,RU,2024-04-02,1507116


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City"
)

# Display the map
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= 50 ) & \
                                       (city_data_df["Max Temp"] >= 20) &  
                                       (city_data_df["Cloudiness"] == 0) &
                                       (city_data_df["Wind Speed"] <= 4.5)]          
                                        
preferred_cities_df.head(20)
# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City ID.1
16,16,fort bragg,35.1390,-79.0060,26.40,54,0,4.12,US,2024-04-02,6941080
35,35,mahina,-17.5065,-149.4890,30.87,66,0,3.09,PF,2024-04-02,4034307
38,38,arraial do cabo,-22.9661,-42.0278,22.98,94,0,2.06,BR,2024-04-02,3471451
90,90,alice springs,-23.7000,133.8833,20.68,42,0,4.12,AU,2024-04-02,2077895
142,142,copiapo,-27.3667,-70.3333,21.11,58,0,2.13,CL,2024-04-02,3893656
253,253,portland,45.5234,-122.6762,23.71,38,0,2.68,US,2024-04-02,5746545
268,268,greenville,35.6127,-77.3663,26.97,53,0,3.60,US,2024-04-02,4469160
347,347,cloverdale,38.8055,-123.0172,22.76,42,0,2.68,US,2024-04-02,5338104
357,357,fortaleza,-3.7227,-38.5247,29.07,79,0,2.06,BR,2024-04-02,6320062
368,368,emerald,-23.5333,148.1667,26.86,67,0,3.86,AU,2024-04-02,2167426


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df = preferred_cities_df[["City", "Country", "Humidity", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(20)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City,Country,Humidity,Lat,Lng,Hotel Name
16,fort bragg,US,54,35.1390,-79.0060,
35,mahina,PF,66,-17.5065,-149.4890,
38,arraial do cabo,BR,94,-22.9661,-42.0278,
90,alice springs,AU,42,-23.7000,133.8833,
142,copiapo,CL,58,-27.3667,-70.3333,
253,portland,US,38,45.5234,-122.6762,
268,greenville,US,53,35.6127,-77.3663,
347,cloverdale,US,42,38.8055,-123.0172,
357,fortaleza,BR,79,-3.7227,-38.5247,
368,emerald,AU,67,-23.5333,148.1667,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius =  20000
limit =1
params = {"categories": "accommodation.hotel",
          "limit" : 1,
          "apiKey": geoapify_key}
    

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = hotel_df.loc[index,'Lat']
    Lng = hotel_df.loc[index,'Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"


    # Add the latitude and longitude to location key for the params dictionary.
   # params["location"] = f"{Lat},{Lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
   
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    
        except (KeyError, IndexError):
        
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data



hotel_df


Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
mahina - nearest hotel: Motu Martin
an exception occured: 'name'
arraial do cabo - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
copiapo - nearest hotel: Hotel Diego de Almeida
portland - nearest hotel: the Hoxton
greenville - nearest hotel: Quality Inn
cloverdale - nearest hotel: Cloverdale Wine Country Inn and Suites
fortaleza - nearest hotel: Hotel Catedral
emerald - nearest hotel: Emerald Central Hotel
morristown - nearest hotel: Days Inn
al khankah - nearest hotel: كونكورد السلام
tamanrasset - nearest hotel: فندق الأمان
murzuq - nearest hotel: فندق باريس - مرزق


,City,Country,Humidity,Lat,Lng,Hotel Name
16,fort bragg,US,54,35.1390,-79.0060,Airborne Inn Lodging
35,mahina,PF,66,-17.5065,-149.4890,Motu Martin
38,arraial do cabo,BR,94,-22.9661,-42.0278,No hotel found
90,alice springs,AU,42,-23.7000,133.8833,Aurora Alice Springs
142,copiapo,CL,58,-27.3667,-70.3333,Hotel Diego de Almeida
253,portland,US,38,45.5234,-122.6762,the Hoxton
268,greenville,US,53,35.6127,-77.3663,Quality Inn
347,cloverdale,US,42,38.8055,-123.0172,Cloverdale Wine Country Inn and Suites
357,fortaleza,BR,79,-3.7227,-38.5247,Hotel Catedral
368,emerald,AU,67,-23.5333,148.1667,Emerald Central Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display
# Need to filter out cities with "No hotel found" and test result
hotel_found_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Configure the map plot
city_hotel_map = hotel_found_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    scale = 1.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
city_hotel_map

# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)